# Análisis Exploratorio de Datos (EDA)

**Proyecto:** Pipeline MLOps - Predicción de Pago a Tiempo de Créditos

Este notebook realiza el análisis exploratorio del dataset cargado, combinando una caracterización inicial con un análisis estadístico profundo.

## 1. Descripción general de los datos
- Caracterización de los datos: categóricos, numéricos, ordinales, nominales, dicotómicos, politómicos.
- Revisión de nulos y unificación de su representación.

In [ ]:
# Importación de librerías y configuración
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings

warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (10, 6)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## 2 & 3. Carga, Unificación y Exploración Inicial
Cargamos los datos, unificamos nulos y eliminamos variables irrelevantes.

In [ ]:
# Cargar datos
try:
    df = pd.read_excel('../../Base_de_datos.xlsx')
    print("✓ Datos cargados desde ruta relativa principal")
except FileNotFoundError:
    try:
        df = pd.read_excel('Base_de_datos.xlsx')
        print("✓ Datos cargados desde directorio actual")
    except FileNotFoundError:
        print("❌ No se encontró el archivo. Verifique la ruta.")

df_original = df.copy()
print(f"Dimensiones: {df.shape[0]:,} filas x {df.shape[1]} columnas")
df.head()

## 4. Información general y estadística descriptiva
Revisión general y estadística descriptiva enriquecida (skewness, kurtosis).

In [ ]:
print("INFORMACIÓN GENERAL:")
df.info()

print("\nESTADÍSTICA DESCRIPTIVA (Numéricas):")
display(df.describe().T)

# Identificar tipos de variables
col_num = df.select_dtypes(include=['number']).columns.tolist()
col_cat = df.select_dtypes(include=['object', 'category']).columns.tolist()
col_date = [c for c in df.columns if 'fecha' in c.lower()]

print(f"\nNuméricas: {len(col_num)} | Categóricas: {len(col_cat)} | Fechas: {len(col_date)}")

## 5. Revisión y tratamiento de valores nulos
Análisis detallado de valores nulos y su impacto.

In [ ]:
# Unificar nulos
nulos_list = ['NA', 'N/A', '', 'null', 'None', 'nan', '?', '-']
df.replace(nulos_list, np.nan, inplace=True)

# Análisis detallado
null_counts = df.isnull().sum()
null_pct = (df.isnull().sum() / len(df)) * 100
null_df = pd.DataFrame({'Nulos': null_counts, 'Porcentaje': null_pct}).sort_values('Nulos', ascending=False)

print("COLUMNAS CON VALORES NULOS:")
print(null_df[null_df['Nulos'] > 0])

plt.figure(figsize=(12, 6))
sns.heatmap(df.isnull(), cbar=False, cmap='Reds')
plt.title('Mapa de Calor de Valores Nulos (Rojo = Nulo)')
plt.show()

## 6. Análisis Univariable (Categóricas y Numéricas)
Distribución detallada de las variables.

In [ ]:
# Categóricas
for col in col_cat:
    if df[col].nunique() > 20: continue # Skip high cardinality
    
    fig, ax = plt.subplots(1, 2, figsize=(14, 5))
    
    # Bar Plot
    count_data = df[col].value_counts()
    count_data.plot(kind='bar', ax=ax[0], color=sns.color_palette('husl'))
    ax[0].set_title(f'Frecuencia: {col}')
    ax[0].tick_params(axis='x', rotation=45)
    
    # Pie Chart
    ax[1].pie(count_data, labels=count_data.index, autopct='%1.1f%%', startangle=90)
    ax[1].set_title(f'Proporción: {col}')
    plt.tight_layout()
    plt.show()

## 7. Visualización Numérica y Detección de Outliers
Histogramas y Boxplots para entender la distribución y detectar anomalías.

In [ ]:
# Numéricas & Outliers
for col in col_num:
    if 'id' in col.lower(): continue
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Histograma con KDE
    sns.histplot(df[col].dropna(), kde=True, ax=axes[0])
    axes[0].set_title(f'Distribución: {col}')
    
    # Boxplot
    sns.boxplot(x=df[col].dropna(), ax=axes[1])
    axes[1].set_title(f'Outliers: {col}')
    
    plt.tight_layout()
    plt.show()
    
    # IQR Outlier Detection
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df[(df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))]
    if len(outliers) > 0:
        print(f"⚠️ {col}: {len(outliers)} outliers detectados ({len(outliers)/len(df)*100:.1f}%)")

## 8. Análisis Avanzado (Bivariable y Multivariable)
Relaciones entre variables y la variable objetivo `Pago_atiempo`.

In [ ]:
target = 'Pago_atiempo'

# Correlación
if len(col_num) > 1:
    plt.figure(figsize=(12, 10))
    corr = df[col_num].corr()
    mask = np.triu(np.ones_like(corr, dtype=bool))
    sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', center=0)
    plt.title('Matriz de Correlación')
    plt.show()

# Relación con Target
if target in df.columns:
    print(f"CORRELACIÓN CON {target.upper()}:")
    print(df[col_num].corrwith(df[target]).sort_values(ascending=False))

    # Boxplots por target
    for col in col_num:
        if col == target: continue
        if 'id' in col.lower(): continue
        
        plt.figure(figsize=(8, 4))
        sns.boxplot(x=target, y=col, data=df)
        plt.title(f'{col} vs {target}')
        plt.show()